In [1]:
import numpy as np
import scipy as sc
import matplotlib.pyplot as plt
import scipy.optimize as optimize
from iminuit import Minuit
import pandas as pd
import scipy.interpolate as interpolate
from scipy.integrate import quad
from matplotlib import colors

In [2]:
def chisq(obs, exp, error):
    return np.sum(np.square(obs - exp) / np.square(error))

In [3]:
def Gauss(E, A, mu, sigma):
    return A * (np.exp(-1/2 * np.square((E - mu) / sigma)) / (sigma * np.sqrt(2*np.pi)))

def Gauss_int(A, mu, sigma, Em, Ep):
    return quad(Gauss, Em, Ep, args=(A, mu, sigma))

As we will do a multiple broken power law, from now on the position of the knots will not be a free parameter anymore in order to make the convergence of the fit easier.

In [4]:
def multiple_bpl(xdata, params):
    if knots < 3 or knots > 30: #change this when adding more number of knots
        raise Exception('knots have to be larger or equal than 3 and smaller than 30')
    else:
        polw = np.zeros(len(xdata))
        Es = np.geomspace(xdata[0], xdata[-1], knots)
        lam = np.zeros(knots-1)
        phi = np.zeros(knots-1)
        phi_0 = params[0] #len(sqrtdelta_lam) = len(lam)-1 = len(phi)-1
        lam0 = params[1]
        sqrtdelta_lam = params[2:knots]
        lam[0] = lam0
        phi[0] = phi_0 * ((1/Es[1]) ** (-lam[0]))
        for i in range(1, knots-1):
            lam[i] = lam[i-1] + ((sqrtdelta_lam[i-1]) ** 2)
            phi[i] = phi[i-1] * ((1/Es[i]) ** (-(sqrtdelta_lam[i-1]) ** 2))
        for i in range(knots-1):
            for j in range(len(xdata)):
                if Es[i] < xdata[j] <= Es[i+1]:
                    polw[j] = phi[i] * ((xdata[j]) ** (-lam[i]))
                elif xdata[j] <= Es[1]:
                    polw[j] = phi[0] * ((xdata[j]) ** (-lam[0]))
                elif xdata[j] > Es[-1]:
                    polw[j] = phi[-2] * ((xdata[j]) ** (-lam[-2]))
    return polw


In [5]:
# least-squares score function = sum of data residuals squared
def LSQ(params):
    return np.sum((ydata - multiple_bpl(xdata, params)) ** 2 / ydata_u ** 2)

Now we define the function of the fit

In [6]:
def multiple_bpl_fit(initial_guess):
    #print('Warning: the shape of the parametes will depend on the number of knots (defined as knots)')
    #print('Warning: this will give the parameters of the fit and its errors which have to be put into the multiple_bpl function in order to get the fit')
    LSQ.errordef = Minuit.LIKELIHOOD

    m = Minuit(LSQ, initial_guess)
    #m.limits["initial_guess[2:-1]"] = (0,None)
    m.tol = 1e-20

    m.migrad()
    print(m.values)

    m.hesse()
    print(m.errors)

    return m

Let's check that the program works as intended

In [7]:
file = np.loadtxt('tau_dominguez11.out')

pdfile = pd.DataFrame(file)
pdfile = pdfile.rename(columns={ 0 : 'E [TeV]', 1: 'tau z=0.01', 2: 'tau z=0.02526316', 3: 'tau z=0.04052632', 4: 'tau z=0.05578947', 5: 'tau z=0.07105263', 6: 'tau z=0.08631579', 7: 'tau z=0.10157895', 8: 'tau z=0.11684211', 9: 'tau z=0.13210526', 10: 'tau z=0.14736842', 11: 'tau z=0.16263158', 12: 'tau z=0.17789474', 13: 'tau z=0.19315789', 14: 'tau z=0.20842105'})

E = pdfile['E [TeV]'].to_numpy()
tau = pdfile['tau z=0.20842105'].to_numpy()
logE = np.log10(E)

K =  8.70e-6 #m−2  s−1  TeV−1
dNdE = K / ((E/0.25)**2.03) 
m_tau = -tau
dNdEa = dNdE * np.exp(m_tau)
SED = np.square(E) * dNdEa
SED_u = SED * 0.1

Now let's try it with more realistic data. For that, we will copy the data from the Aeff_bkg_spline.ipynb

In [8]:
fileEA = np.loadtxt('EffArea50h.txt', skiprows = 11)
EffA = pd.DataFrame(fileEA)
EffA = EffA.rename(columns={ 0 : 'E [TeV]', 1: 'Eff area (m^2)'})
EffaE= EffA['E [TeV]'].to_numpy()
Effa = EffA['Eff area (m^2)'].to_numpy()

#absorption file
file = np.loadtxt('tau_dominguez11.out')
pdfile = pd.DataFrame(file)
pdfile = pdfile.rename(columns={ 0 : 'E [TeV]', 1: 'tau z=0.01', 2: 'tau z=0.02526316', 3: 'tau z=0.04052632', 4: 'tau z=0.05578947', 5: 'tau z=0.07105263', 6: 'tau z=0.08631579', 7: 'tau z=0.10157895', 8: 'tau z=0.11684211', 9: 'tau z=0.13210526', 10: 'tau z=0.14736842', 11: 'tau z=0.16263158', 12: 'tau z=0.17789474', 13: 'tau z=0.19315789', 14: 'tau z=0.20842105'})
E = pdfile['E [TeV]'].to_numpy()
tau = pdfile['tau z=0.20842105'].to_numpy()

In [9]:
fileBkg = np.loadtxt('Bkg50h.txt', skiprows = 10)
Bkgpd = pd.DataFrame(fileBkg)
Bkgpd = Bkgpd.rename(columns={ 0 : 'E_min (TeV)', 1: 'E_max (TeV)', 2: 'Bck Rate (Hz/deg^2)'})

BckgEmin = Bkgpd['E_min (TeV)'].to_numpy()
BckgEmax = Bkgpd['E_max (TeV)'].to_numpy()
Bckg = Bkgpd['Bck Rate (Hz/deg^2)'].to_numpy()
BckgEdiff = BckgEmax - BckgEmin

BckgElogmean = []
for i in range(len(Bckg)):
    BckgElogmean.append(10**(np.mean([np.log10(BckgEmax[i]), np.log10(BckgEmin[i])])))


Ebinsmin = np.zeros(len(E))
Ebinsmax = np.zeros(len(E))

for i in range(1, len(E)):
    Ebinsmin[i] = 10**(np.mean([np.log10(E[i]), np.log10(E[i-1])]))
for i in range(len(E) - 1):
    Ebinsmax[i] = 10**(np.mean([np.log10(E[i]), np.log10(E[i+1])]))
    
Ebinsmin[0] = 10**(np.log10(Ebinsmin[1])-(np.log10(Ebinsmin[2])-np.log10(Ebinsmin[1])))
Ebinsmax[len(E)-1] = 10**(np.log10(Ebinsmax[len(E)-2])-(np.log10(Ebinsmax[len(E)-3])-np.log10(Ebinsmax[len(E)-2])))
Ebinsw = Ebinsmax - Ebinsmin

In [10]:
DifBckg = Bckg / (BckgEmax - BckgEmin) #Hz/deg**2
def log_interp1d(xx, yy):
    logx = np.log10(xx)
    logy = np.log10(yy)
    interp = interpolate.interp1d(logx, logy, fill_value='extrapolate', kind='slinear')
    log_interp = lambda zz: np.power(10.0, interp(np.log10(zz)))
    return log_interp

interpolation = log_interp1d(BckgElogmean,DifBckg)

In [11]:
def trapint(toplim, bottomlim):
    return ((toplim - bottomlim) * (interpolation(toplim) + interpolation(bottomlim)) / 2)

def sqint(toplim, bottomlim):
    return ((toplim - bottomlim) * (interpolation((toplim + bottomlim) / 2)))


In [12]:
newBckgrate = np.zeros([len(E[2:37]),2])
for i in range(len(E[2:37])):
    newBckgrate[i] = quad(interpolation, Ebinsmin[i+2], Ebinsmax[i+2])
newBckgrate = newBckgrate[:,0]

In [13]:
#“photon index  = 2.03  +/- 0.03 and normalization factor at 250 GeV    f0 = (8.70 +/- 0.17) * 10^-10 cm^-2  s^-1  TeV^-1.”
K =  8.70e-6 #m−2  s−1  TeV−1
dNdE = K / ((E/0.25)**2.03) 
m_tau = -tau
dNdEa = dNdE * np.exp(m_tau)

In [14]:
EffaA2 = np.zeros(len(E[2:37]))
for i in range(len(E[2:37])):
    for j in range(len(EffaE[9:51])):
        if EffaE[j+9] < E[i+2] and EffaE[j+10] > E[i+2]:
            m = (Effa[j+10]-Effa[j+9])/(EffaE[j+10]-EffaE[j+9])
            n = Effa[j+9] - m * EffaE[j+9]
            EffaA2[i] = m * E[i+2] + n

In [15]:
fileAng = np.loadtxt('Angres.txt', skiprows = 11)
Angresall = pd.DataFrame(fileAng)
Angresall = Angresall.rename(columns={ 0 : 'E [TeV]', 1: 'Angular resolution (deg)'})
AngresE= Angresall['E [TeV]'].to_numpy()
Angres = Angresall['Angular resolution (deg)'].to_numpy()

logAngres = np.log10(Angres)
logAngresE = np.log10(AngresE)
logE = np.log10(E)


logAngres2 = np.zeros(len(E[2:37]))
for i in range(len(E[2:37])):
    for j in range(len(AngresE[1:-1])):
        if logAngresE[j+1] < logE[i+2] and logAngresE[j+2] > logE[i+2]:
            m = (logAngres[j+2]-logAngres[j+1])/(logAngresE[j+2]-logAngresE[j+1])
            n = logAngres[j+1] - m * logAngresE[j+1]
            logAngres2[i] = m * logE[i+2] + n
Angres2 = 10**logAngres2

In [16]:
def degtorad(ang):
    return ang * np.pi / 180


time = 42480 #42480 #180000#s
skyang = (180/np.pi)**2 * 2 * np.pi * (1-np.cos(degtorad(Angres2)))#deg^2

bckgmu = newBckgrate * time * skyang

Let's load the Energy resolution

In [17]:
fileEres = np.loadtxt('Eres.txt', skiprows = 8)
Eresall = pd.DataFrame(fileEres)
Eresall = Eresall.rename(columns={ 0 : 'E [TeV]', 1: 'Energy resolution (deg)'})
EresE = Eresall['E [TeV]'].to_numpy()
Eres = Eresall['Energy resolution (deg)'].to_numpy()

logEres = np.log10(Eres)
logEresE = np.log10(EresE)
logE = np.log10(E)


logEres2 = np.zeros(len(E[2:37]))
for i in range(len(E[2:37])):
    for j in range(len(EresE[0:-1])):
        if logEresE[j] < logE[i+3] and logEresE[j+1] > logE[i+3]:
            m = (logEres[j+1]-logEres[j])/(logEresE[j+1]-logEresE[j])
            n = logEres[j] - m * logEresE[j]
            logEres2[i] = m * logE[i+3] + n
Eres2 = 10**logEres2
Eres3 = Eres2 * E[2:37]


logEbins = np.zeros(len(E[2:37])+1)
for i in range(len(E[2:37])+1):
    if i == 0:
        logEbins[i] = logE[2] - ((logE[3]-logE[2])/2)
    elif i == (len(E[2:37])):
        logEbins[i] = logE[2:37][i-1] + ((logE[2:37][i-1]-logE[2:37][i-2])/2)
    else:
        logEbins[i] = (logE[2:37][i] + logE[2:37][i-1]) / 2

Ebins = 10 ** logEbins

mu_vec = dNdEa[2:37] * EffaA2 * Ebinsw[2:37] * time

mu_vec_reco = np.zeros(len(mu_vec))
mu_vec_i = np.zeros(len(mu_vec))

for i in range(len(mu_vec)):
    for j in range(len(mu_vec)):
        A = mu_vec[i]
        B = bckgmu[i]
        Em = Ebins[j]
        Ep = Ebins[j+1]
        sigma = Eres3[i]
        mu = E[2:37][i]

        mu_vec_i[j] = Gauss_int(A, mu, sigma, Em, Ep)[0]
    mu_vec_reco = mu_vec_reco + mu_vec_i

In [18]:
niter = 1000
chisqs_iter = []
alphas_iter = []

for j in range(niter):
    my_generator = np.random.default_rng(101)
    Simbckg1 = my_generator.poisson(bckgmu)
    # Simbckg1 = Simbckg1.astype(float)
    # for i in range(len(Simbckg1)):
    #     if Simbckg1[i] == 0:
    #         Simbckg1[i] = bckgmu[i]
    Simbckg1_u = np.sqrt(Simbckg1)
    Simbckg5 = my_generator.poisson(5*bckgmu)/5
    # Simbckg5 = Simbckg5.astype(float)
    # for i in range(len(Simbckg5)):
    #     if Simbckg5[i] == 0:
    #         Simbckg5[i] = bckgmu[i]
    Simbckg5_u = np.sqrt(Simbckg5)

    N = my_generator.poisson(mu_vec_reco)

    for i in range(len(N)):
        if N[i] == 0:
            N[i] = 1
    N_u = np.sqrt(N)

    NpB = np.around(N + Simbckg1 - Simbckg5).astype(int)
    NpB_u = N_u + Simbckg1_u - Simbckg5_u

    dNdE_b = NpB[1:] / EffaA2[1:35] / Ebinsw[3:37] / time
    dNdE_b_u = NpB_u[1:] / EffaA2[1:35] / Ebinsw[3:37] / time

    SED = np.square(E[3:37]) * dNdE_b
    SED_u = np.square(E[3:37]) * dNdE_b_u

    chisqs = []
    alphas = np.arange(0,2.05,0.05)

    for i in range(41):
        alpha = alphas[i]
        print("alpha = ", alpha)
        dNdE2 = dNdE_b[4:] * np.exp(alpha*tau[7:37])
        dNdE2_u = dNdE_b_u[4:] * np.exp(alpha*tau[7:37])
        SED2 = np.square(E[7:37]) * dNdE2
        SED_u2 = np.square(E[7:37]) * dNdE2_u
        ydata = SED2
        xdata = E[7:37]
        knots = 5
        ydata_u = SED_u2
        initial_guess = np.array([5e-7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]) #phi_0, lam1, deltas (len(deltas)=knots-2))
        things = multiple_bpl_fit(initial_guess=initial_guess)
        chi2 = chisq(SED2, multiple_bpl(E[7:37], things.values), SED_u2)
        chisqs.append(chi2)
        # if alpha == 0. or alpha == 0.15 or alpha == 0.9 or alpha == 1.:
        #     plt.plot(xdata, multiple_bpl(xdata, things.values), label='concave fit', color = 'orange')
        #     plt.errorbar(xdata, ydata, label='original',yerr = ydata_u, capsize=3, fmt='o', alpha=0.8)
        #     plt.xscale('log')
        #     plt.yscale('log')
        #     plt.xlabel(r"$E$ ($TeV$)")
        #     plt.ylabel(r"$E^2$ $dF/dE$ ($TeV$ $m^{-2}$ $s^{-1}$)")
        #     plt.legend()
        #     plt.show()
    alphas_iter.append(alphas)
    chisqs_iter.append(chisqs)
    print("\033[2;31;43m iteration {iter} of {niter} finished \033[0;0m".format(iter = j+1, niter = niter))

alpha =  0.0
<ValueView x0=4.5794578365926226e-07 x1=0.17496730611039862 x2=-1.1220213896723372 x3=0.0 x4=-0.9621740244658725 x5=0.0 x6=0.0 x7=0.0 x8=0.0 x9=0.0 x10=0.0>
<ErrorView x0=0.0 x1=0.0 x2=0.0 x3=0.0 x4=0.0 x5=0.0 x6=0.0 x7=0.0 x8=0.0 x9=0.0 x10=0.0>
alpha =  0.05
<ValueView x0=4.672396857154381e-07 x1=0.15257178422906378 x2=1.119634427463934 x3=0.0 x4=0.0 x5=0.0 x6=0.0 x7=0.0 x8=0.0 x9=0.0 x10=0.0>
<ErrorView x0=0.0 x1=0.0 x2=0.0 x3=0.0 x4=0.0 x5=0.0 x6=0.0 x7=0.0 x8=0.0 x9=0.0 x10=0.0>
alpha =  0.1
<ValueView x0=4.7219829610552465e-07 x1=0.14314973766913136 x2=-1.0925643119974935 x3=0.0 x4=0.0 x5=0.0 x6=0.0 x7=0.0 x8=0.0 x9=0.0 x10=0.0>
<ErrorView x0=0.0 x1=0.0 x2=0.0 x3=0.0 x4=0.0 x5=0.0 x6=0.0 x7=0.0 x8=0.0 x9=0.0 x10=0.0>
alpha =  0.15000000000000002
<ValueView x0=4.635888434265085e-07 x1=0.17300724573962728 x2=-0.9684594422592874 x3=-0.6897778079713662 x4=-0.26949972151813145 x5=0.0 x6=0.0 x7=0.0 x8=0.0 x9=0.0 x10=0.0>
<ErrorView x0=0.0 x1=0.0 x2=0.0 x3=0.0 x4=0.0 x5=0.0

<ErrorView x0=0.0 x1=0.0 x2=0.0 x3=0.0 x4=0.0 x5=0.0 x6=0.0 x7=0.0 x8=0.0 x9=0.0 x10=0.0>
alpha =  1.9500000000000002
<ValueView x0=8.476925859911661e-07 x1=-0.5647453573280539 x2=0.0 x3=0.0 x4=0.0 x5=0.0 x6=0.0 x7=0.0 x8=0.0 x9=0.0 x10=0.0>
<ErrorView x0=0.0 x1=0.0 x2=0.0 x3=0.0 x4=0.0 x5=0.0 x6=0.0 x7=0.0 x8=0.0 x9=0.0 x10=0.0>
alpha =  2.0
<ValueView x0=8.636423890631725e-07 x1=-0.5881445090413687 x2=0.0 x3=0.0 x4=0.0 x5=0.0 x6=0.0 x7=0.0 x8=0.0 x9=0.0 x10=0.0>
<ErrorView x0=0.0 x1=0.0 x2=0.0 x3=0.0 x4=0.0 x5=0.0 x6=0.0 x7=0.0 x8=0.0 x9=0.0 x10=0.0>
 iteration 1 of 1000 finished 
alpha =  0.0
<ValueView x0=4.5794578365926226e-07 x1=0.17496730611039862 x2=-1.1220213896723372 x3=0.0 x4=-0.9621740244658725 x5=0.0 x6=0.0 x7=0.0 x8=0.0 x9=0.0 x10=0.0>
<ErrorView x0=0.0 x1=0.0 x2=0.0 x3=0.0 x4=0.0 x5=0.0 x6=0.0 x7=0.0 x8=0.0 x9=0.0 x10=0.0>
alpha =  0.05
<ValueView x0=4.672396857154381e-07 x1=0.15257178422906378 x2=1.119634427463934 x3=0.0 x4=0.0 x5=0.0 x6=0.0 x7=0.0 x8=0.0 x9=0.0 x10=0.0

<ValueView x0=8.476925859911661e-07 x1=-0.5647453573280539 x2=0.0 x3=0.0 x4=0.0 x5=0.0 x6=0.0 x7=0.0 x8=0.0 x9=0.0 x10=0.0>
<ErrorView x0=0.0 x1=0.0 x2=0.0 x3=0.0 x4=0.0 x5=0.0 x6=0.0 x7=0.0 x8=0.0 x9=0.0 x10=0.0>
alpha =  2.0
<ValueView x0=8.636423890631725e-07 x1=-0.5881445090413687 x2=0.0 x3=0.0 x4=0.0 x5=0.0 x6=0.0 x7=0.0 x8=0.0 x9=0.0 x10=0.0>
<ErrorView x0=0.0 x1=0.0 x2=0.0 x3=0.0 x4=0.0 x5=0.0 x6=0.0 x7=0.0 x8=0.0 x9=0.0 x10=0.0>
 iteration 2 of 1000 finished 
alpha =  0.0
<ValueView x0=4.5794578365926226e-07 x1=0.17496730611039862 x2=-1.1220213896723372 x3=0.0 x4=-0.9621740244658725 x5=0.0 x6=0.0 x7=0.0 x8=0.0 x9=0.0 x10=0.0>
<ErrorView x0=0.0 x1=0.0 x2=0.0 x3=0.0 x4=0.0 x5=0.0 x6=0.0 x7=0.0 x8=0.0 x9=0.0 x10=0.0>
alpha =  0.05
<ValueView x0=4.672396857154381e-07 x1=0.15257178422906378 x2=1.119634427463934 x3=0.0 x4=0.0 x5=0.0 x6=0.0 x7=0.0 x8=0.0 x9=0.0 x10=0.0>
<ErrorView x0=0.0 x1=0.0 x2=0.0 x3=0.0 x4=0.0 x5=0.0 x6=0.0 x7=0.0 x8=0.0 x9=0.0 x10=0.0>
alpha =  0.1
<ValueView x0

<ErrorView x0=0.0 x1=0.0 x2=0.0 x3=0.0 x4=0.0 x5=0.0 x6=0.0 x7=0.0 x8=0.0 x9=0.0 x10=0.0>
alpha =  1.55
<ValueView x0=7.209950356248315e-07 x1=-0.34986598250396367 x2=0.0 x3=0.0 x4=0.0 x5=0.0 x6=0.0 x7=0.0 x8=0.0 x9=0.0 x10=0.0>
<ErrorView x0=0.0 x1=0.0 x2=0.0 x3=0.0 x4=0.0 x5=0.0 x6=0.0 x7=0.0 x8=0.0 x9=0.0 x10=0.0>
alpha =  1.6
<ValueView x0=7.36774147864857e-07 x1=-0.37974640430539625 x2=0.0 x3=0.0 x4=0.0 x5=0.0 x6=0.0 x7=0.0 x8=0.0 x9=0.0 x10=0.0>
<ErrorView x0=0.0 x1=0.0 x2=0.0 x3=0.0 x4=0.0 x5=0.0 x6=0.0 x7=0.0 x8=0.0 x9=0.0 x10=0.0>
alpha =  1.6500000000000001
<ValueView x0=7.525670225461897e-07 x1=-0.4087054468254109 x2=0.0 x3=0.0 x4=0.0 x5=0.0 x6=0.0 x7=0.0 x8=0.0 x9=0.0 x10=0.0>
<ErrorView x0=0.0 x1=0.0 x2=0.0 x3=0.0 x4=0.0 x5=0.0 x6=0.0 x7=0.0 x8=0.0 x9=0.0 x10=0.0>
alpha =  1.7000000000000002
<ValueView x0=7.683737219796038e-07 x1=-0.4367668559525443 x2=0.0 x3=0.0 x4=0.0 x5=0.0 x6=0.0 x7=0.0 x8=0.0 x9=0.0 x10=0.0>
<ErrorView x0=0.0 x1=0.0 x2=0.0 x3=0.0 x4=0.0 x5=0.0 x6=0.0 

<ErrorView x0=0.0 x1=0.0 x2=0.0 x3=0.0 x4=0.0 x5=0.0 x6=0.0 x7=0.0 x8=0.0 x9=0.0 x10=0.0>
alpha =  1.1
<ValueView x0=5.817313014051057e-07 x1=-0.045801797409115365 x2=0.0 x3=0.0 x4=-0.7380902621454228 x5=0.0 x6=0.0 x7=0.0 x8=0.0 x9=0.0 x10=0.0>
<ErrorView x0=0.0 x1=0.0 x2=0.0 x3=0.0 x4=0.0 x5=0.0 x6=0.0 x7=0.0 x8=0.0 x9=0.0 x10=0.0>
alpha =  1.1500000000000001
<ValueView x0=5.956071850533073e-07 x1=-0.07664889685481087 x2=0.0 x3=0.0 x4=0.0 x5=0.0 x6=0.0 x7=0.0 x8=0.0 x9=0.0 x10=0.0>
<ErrorView x0=0.0 x1=0.0 x2=0.0 x3=0.0 x4=0.0 x5=0.0 x6=0.0 x7=0.0 x8=0.0 x9=0.0 x10=0.0>
alpha =  1.2000000000000002
<ValueView x0=6.111509134610111e-07 x1=-0.11408587894077006 x2=0.0 x3=0.0 x4=0.0 x5=0.0 x6=0.0 x7=0.0 x8=0.0 x9=0.0 x10=0.0>
<ErrorView x0=0.0 x1=0.0 x2=0.0 x3=0.0 x4=0.0 x5=0.0 x6=0.0 x7=0.0 x8=0.0 x9=0.0 x10=0.0>
alpha =  1.25
<ValueView x0=6.26744889390822e-07 x1=-0.15061006035476196 x2=0.0 x3=0.0 x4=0.0 x5=0.0 x6=0.0 x7=0.0 x8=0.0 x9=0.0 x10=0.0>
<ErrorView x0=0.0 x1=0.0 x2=0.0 x3=0.0 x4

...

<ErrorView x0=0.0 x1=0.0 x2=0.0 x3=0.0 x4=0.0 x5=0.0 x6=0.0 x7=0.0 x8=0.0 x9=0.0 x10=0.0>
alpha =  1.6
<ValueView x0=7.36774147864857e-07 x1=-0.37974640430539625 x2=0.0 x3=0.0 x4=0.0 x5=0.0 x6=0.0 x7=0.0 x8=0.0 x9=0.0 x10=0.0>
<ErrorView x0=0.0 x1=0.0 x2=0.0 x3=0.0 x4=0.0 x5=0.0 x6=0.0 x7=0.0 x8=0.0 x9=0.0 x10=0.0>
alpha =  1.6500000000000001
<ValueView x0=7.525670225461897e-07 x1=-0.4087054468254109 x2=0.0 x3=0.0 x4=0.0 x5=0.0 x6=0.0 x7=0.0 x8=0.0 x9=0.0 x10=0.0>
<ErrorView x0=0.0 x1=0.0 x2=0.0 x3=0.0 x4=0.0 x5=0.0 x6=0.0 x7=0.0 x8=0.0 x9=0.0 x10=0.0>
alpha =  1.7000000000000002
<ValueView x0=7.683737219796038e-07 x1=-0.4367668559525443 x2=0.0 x3=0.0 x4=0.0 x5=0.0 x6=0.0 x7=0.0 x8=0.0 x9=0.0 x10=0.0>
<ErrorView x0=0.0 x1=0.0 x2=0.0 x3=0.0 x4=0.0 x5=0.0 x6=0.0 x7=0.0 x8=0.0 x9=0.0 x10=0.0>
alpha =  1.75
<ValueView x0=7.841954328130048e-07 x1=-0.4639592808057523 x2=0.0 x3=0.0 x4=0.0 x5=0.0 x6=0.0 x7=0.0 x8=0.0 x9=0.0 x10=0.0>
<ErrorView x0=0.0 x1=0.0 x2=0.0 x3=0.0 x4=0.0 x5=0.0 x6=0.0 x

<ErrorView x0=0.0 x1=0.0 x2=0.0 x3=0.0 x4=0.0 x5=0.0 x6=0.0 x7=0.0 x8=0.0 x9=0.0 x10=0.0>
alpha =  1.1500000000000001
<ValueView x0=5.956071850533073e-07 x1=-0.07664889685481087 x2=0.0 x3=0.0 x4=0.0 x5=0.0 x6=0.0 x7=0.0 x8=0.0 x9=0.0 x10=0.0>
<ErrorView x0=0.0 x1=0.0 x2=0.0 x3=0.0 x4=0.0 x5=0.0 x6=0.0 x7=0.0 x8=0.0 x9=0.0 x10=0.0>
alpha =  1.2000000000000002
<ValueView x0=6.111509134610111e-07 x1=-0.11408587894077006 x2=0.0 x3=0.0 x4=0.0 x5=0.0 x6=0.0 x7=0.0 x8=0.0 x9=0.0 x10=0.0>
<ErrorView x0=0.0 x1=0.0 x2=0.0 x3=0.0 x4=0.0 x5=0.0 x6=0.0 x7=0.0 x8=0.0 x9=0.0 x10=0.0>
alpha =  1.25
<ValueView x0=6.26744889390822e-07 x1=-0.15061006035476196 x2=0.0 x3=0.0 x4=0.0 x5=0.0 x6=0.0 x7=0.0 x8=0.0 x9=0.0 x10=0.0>
<ErrorView x0=0.0 x1=0.0 x2=0.0 x3=0.0 x4=0.0 x5=0.0 x6=0.0 x7=0.0 x8=0.0 x9=0.0 x10=0.0>
alpha =  1.3
<ValueView x0=6.423815868818887e-07 x1=-0.18620294257715123 x2=0.0 x3=0.0 x4=0.0 x5=0.0 x6=0.0 x7=0.0 x8=0.0 x9=0.0 x10=0.0>
<ErrorView x0=0.0 x1=0.0 x2=0.0 x3=0.0 x4=0.0 x5=0.0 x6=0.

<ValueView x0=5.447254951517891e-07 x1=0.02262314195736171 x2=-0.4719988910791047 x3=-0.34992429704327926 x4=-0.6804438070522902 x5=0.0 x6=0.0 x7=0.0 x8=0.0 x9=0.0 x10=0.0>
<ErrorView x0=0.0 x1=0.0 x2=0.0 x3=0.0 x4=0.0 x5=0.0 x6=0.0 x7=0.0 x8=0.0 x9=0.0 x10=0.0>
alpha =  0.9
<ValueView x0=5.509937391545574e-07 x1=0.011967626130686625 x2=-0.41448123154205446 x3=-0.31223667338037014 x4=-0.7025990190111286 x5=0.0 x6=0.0 x7=0.0 x8=0.0 x9=0.0 x10=0.0>
<ErrorView x0=0.0 x1=0.0 x2=0.0 x3=0.0 x4=0.0 x5=0.0 x6=0.0 x7=0.0 x8=0.0 x9=0.0 x10=0.0>
alpha =  0.9500000000000001
<ValueView x0=5.573287496483784e-07 x1=0.0013224647401458118 x2=-0.3475977371463765 x3=-0.2693512472516748 x4=-0.724391004701696 x5=0.0 x6=0.0 x7=0.0 x8=0.0 x9=0.0 x10=0.0>
<ErrorView x0=0.0 x1=0.0 x2=0.0 x3=0.0 x4=0.0 x5=0.0 x6=0.0 x7=0.0 x8=0.0 x9=0.0 x10=0.0>
alpha =  1.0
<ValueView x0=5.548558586733728e-07 x1=0.01685161936203279 x2=0.0 x3=-0.37212758985904054 x4=-0.6859602592988403 x5=0.0 x6=0.0 x7=0.0 x8=0.0 x9=0.0 x10=0.0

<ValueView x0=4.968965049879161e-07 x1=0.10826910007874468 x2=-0.7944810396225052 x3=-0.568444712347141 x4=-0.4819684817936951 x5=0.0 x6=0.0 x7=0.0 x8=0.0 x9=0.0 x10=0.0>
<ErrorView x0=0.0 x1=0.0 x2=0.0 x3=0.0 x4=0.0 x5=0.0 x6=0.0 x7=0.0 x8=0.0 x9=0.0 x10=0.0>
alpha =  0.5
<ValueView x0=5.026561863852874e-07 x1=0.09752270919784616 x2=-0.7616474453250824 x3=-0.5458038081123082 x4=-0.5097896323026456 x5=0.0 x6=0.0 x7=0.0 x8=0.0 x9=0.0 x10=0.0>
<ErrorView x0=0.0 x1=0.0 x2=0.0 x3=0.0 x4=0.0 x5=0.0 x6=0.0 x7=0.0 x8=0.0 x9=0.0 x10=0.0>
alpha =  0.55
<ValueView x0=5.080069580282131e-07 x1=0.08804097176065676 x2=-0.7214767292282651 x3=-0.5761052858101985 x4=0.0 x5=0.0 x6=0.0 x7=0.0 x8=0.0 x9=0.0 x10=0.0>
<ErrorView x0=0.0 x1=0.0 x2=0.0 x3=0.0 x4=0.0 x5=0.0 x6=0.0 x7=0.0 x8=0.0 x9=0.0 x10=0.0>
alpha =  0.6000000000000001
<ValueView x0=5.143598641701458e-07 x1=0.07606570793354932 x2=-0.6913397758342348 x3=-0.49758494824315086 x4=-0.5621888272436408 x5=0.0 x6=0.0 x7=0.0 x8=0.0 x9=0.0 x10=0.0>
<Er

KeyboardInterrupt: 

In [ ]:
alphas_iter2 = np.concatenate(alphas_iter)
chisqs_iter2 = np.concatenate(chisqs_iter)


fig, ax = plt.subplots()
h = ax.hist2d(alphas_iter2, chisqs_iter2, bins=100, norm = colors.LogNorm(), cmap = "RdYlGn_r")
ax.set_xlabel(r'$\alpha$')
ax.set_ylabel(r'$\chi^2$')
fig.colorbar(h[3], ax=ax, label="number of times")

In [ ]:
niter = 10
chisqs_iter = []
alphas_iter = []

for j in range(niter):
    my_generator = np.random.default_rng(j)
    Simbckg1 = my_generator.poisson(bckgmu)
    # Simbckg1 = Simbckg1.astype(float)
    # for i in range(len(Simbckg1)):
    #     if Simbckg1[i] == 0:
    #         Simbckg1[i] = bckgmu[i]
    Simbckg1_u = np.sqrt(Simbckg1)
    Simbckg5 = my_generator.poisson(5*bckgmu)/5
    # Simbckg5 = Simbckg5.astype(float)
    # for i in range(len(Simbckg5)):
    #     if Simbckg5[i] == 0:
    #         Simbckg5[i] = bckgmu[i]
    Simbckg5_u = np.sqrt(Simbckg5)

    N = my_generator.poisson(mu_vec_reco)

    # for i in range(len(N)):
    #     if N[i] == 0:
    #         N[i] = mu[i]
    N_u = np.sqrt(N)

    NpB = np.around(N + Simbckg1 - Simbckg5).astype(int)
    NpB_u = N_u + Simbckg1_u - Simbckg5_u

    dNdE_b = NpB[1:] / EffaA2[1:35] / Ebinsw[3:37] / time
    dNdE_b_u = NpB_u[1:] / EffaA2[1:35] / Ebinsw[3:37] / time

    SED = np.square(E[3:37]) * dNdE_b
    SED_u = np.square(E[3:37]) * dNdE_b_u

    chisqs = []
    alphas = np.arange(0.76,1.14,0.02)

    for i in range(19):
        alpha = alphas[i]
        print("alpha = ", alpha)
        dNdE2 = dNdE_b[4:] * np.exp(alpha*tau[7:37])
        dNdE2_u = dNdE_b_u[4:] * np.exp(alpha*tau[7:37])
        SED2 = np.square(E[7:37]) * dNdE2
        SED_u2 = np.square(E[7:37]) * dNdE2_u
        ydata = SED2
        xdata = E[7:37]
        knots = 5
        ydata_u = SED_u2
        initial_guess = np.array([5e-7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]) #phi_0, lam1, deltas (len(deltas)=knots-2))
        things = multiple_bpl_fit(initial_guess=initial_guess)
        chi2 = chisq(SED2, multiple_bpl(E[7:37], things.values), SED_u2)
        chisqs.append(chi2)
    alphas_iter.append(alphas)
    chisqs_iter.append(chisqs)
    print("\033[2;31;43m iteration {iter} of {niter} finished \033[0;0m".format(iter = j+1, niter = niter))


In [ ]:
alphas_iter2 = np.concatenate(alphas_iter)
chisqs_iter2 = np.concatenate(chisqs_iter)


fig, ax = plt.subplots()
h = ax.hist2d(alphas_iter2, chisqs_iter2, bins=100, norm = colors.LogNorm(), cmap = "RdYlGn_r")
ax.set_xlabel(r'$\alpha$')
ax.set_ylabel(r'$\chi^2$')
fig.colorbar(h[3], ax=ax, label="number of times")

In [ ]:
# niter = 10
# mu = dNdEa[3:37] * EffaA2 * Ebinsw[3:37] * time 
# chisqs_iter = []
# alphas_iter = []

# if True:
#     my_generator = np.random.default_rng(106)
#     Simbckg1 = my_generator.poisson(bckgmu)
#     # Simbckg1 = Simbckg1.astype(float)
#     # for i in range(len(Simbckg1)):
#     #     if Simbckg1[i] == 0:
#     #         Simbckg1[i] = bckgmu[i]
#     Simbckg1_u = np.sqrt(Simbckg1)
#     Simbckg5 = my_generator.poisson(5*bckgmu)/5
#     # Simbckg5 = Simbckg5.astype(float)
#     # for i in range(len(Simbckg5)):
#     #     if Simbckg5[i] == 0:
#     #         Simbckg5[i] = bckgmu[i]
#     Simbckg5_u = np.sqrt(Simbckg5)

#     N = my_generator.poisson(mu)

#     # for i in range(len(N)):
#     #     if N[i] == 0:
#     #         N[i] = mu[i]
#     N_u = np.sqrt(N)
#     dNdE_p = dNdEa[3:37] * N/mu
#     dNdE_p_u = dNdEa[3:37] * 1/np.sqrt(N)

#     Simbckg = (Simbckg1 - Simbckg5) / EffaA2 / Ebinsw[3:37] / time
#     Simbckg_u = (Simbckg1_u + Simbckg5_u) / EffaA2 / Ebinsw[3:37] / time


#     dNdE_b = dNdE_p + Simbckg
#     dNdE_b_u = dNdE_p_u + Simbckg_u

#     SED = np.square(E[3:37]) * dNdE_b
#     SED_u = np.square(E[3:37]) * dNdE_b_u

#     chisqs = []
#     alphas = np.arange(0.76,1.14,0.02)

#     for i in range(19):
#         alpha = alphas[i]
#         dNdE2 = dNdE_b * np.exp(alpha*tau[3:37])
#         dNdE2_u = dNdE_b_u * np.exp(alpha*tau[3:37])
#         SED2 = np.square(E[3:37]) * dNdE2
#         SED_u2 = np.square(E[3:37]) * dNdE2_u
#         ydata = SED2
#         xdata = E[3:37]
#         knots =5
#         ydata_u = SED_u2
#         initial_guess = np.array([5e-7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]) #phi_0, lam1, deltas (len(deltas)=knots-2))
#         things = multiple_bpl_fit(initial_guess=initial_guess)
#         chi2 = chisq(SED2, multiple_bpl(E[3:37], things.values), SED_u2)
#         chisqs.append(chi2)
#     alphas_iter.append(alphas)
#     chisqs_iter.append(chisqs)
#     print("\033[2;31;43m iteration {iter} of {niter} finished \033[0;0m".format(iter = j+1, niter = niter))

# plt.plot(alphas, chisqs, 'o')
# plt.yscale('log')
# plt.xlabel(r'$\alpha$')
# plt.ylabel(r'$\chi^2$')
# plt.show()